In [ ]:
# riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time          

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.")
print("=" *100)
query_txt = input('1.수집할 자료의 키워드는 무엇입니까? : ')

#Step 3. 크롬 드라이버 설정 및 웹 페이지 열기
s = Service("c:/chromedriver-win64/chromedriver.exe")
driver = webdriver.Chrome(service=s)

url = 'https://www.riss.kr/'
driver.get(url)
time.sleep(5)
driver.maximize_window()

#Step 4. 자동으로 검색어 입력 후 조회하기
element = driver.find_element(By.NAME,'query')
driver.find_element(By.ID,'query').click( )
element.send_keys(query_txt)
element.send_keys("\n")

 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.
1.수집할 자료의 키워드는 무엇입니까? : 전염병


In [10]:
# Step 6.학위 논문 메뉴 선택하기
# 메뉴선택방법 1 - LINK_TEXT( ) 사용하기
driver.find_element(By.LINK_TEXT,'학위논문').click()
time.sleep(2)

# 메뉴 선택방법 2 - XPATH 사용하기
#driver.find_element(By.XPATH , '//*[@id="tabMenu"]/ul/li/div/ul/li[3]/a/span').click()
#time.sleep(2)

In [ ]:
#Step 7.Beautiful Soup 로 본문 내용만 추출하기
from bs4 import BeautifulSoup
html_1 = driver.page_source
soup_1 = BeautifulSoup(html_1, 'html.parser')

# 총 검색 건수를 보여주고 수집할 건수 입력받기
import math
total_cnt = soup_1.find('div','searchBox pd').find('span','num').get_text()
print('검색하신 키워드 %s (으)로 총 %s 건의 학위논문이 검색되었습니다' %(query_txt,total_cnt))
collect_cnt = int(input('이 중에서 몇 건을 수집하시겠습니까?: '))
collect_page_cnt = math.ceil(collect_cnt / 10)
print('%s 건의 데이터를 수집하기 위해 %s 페이지의 게시물을 조회합니다.' %(collect_cnt,collect_page_cnt))
print('=' *80)

#Step 8. 각 항목별로 데이터를 추출하여 리스트에 저장하기
no2 = [ ]        #번호 저장
title2 = [ ]     #논문제목 저장
author2 = [ ]    #논문저자 저장
company2 = [ ]       #소속기관 저장
date2 = [ ]      #발표년도 저장
hak2 = [ ]       #학위정보 저장
no = 1

for a in range(1, collect_page_cnt + 1) :
    
    html_2 = driver.page_source
    soup_2 = BeautifulSoup(html_2, 'html.parser')

    content_2 = soup_2.find('div','srchResultListW').find_all('li')
    
    for i in content_2 :    
        #1. 논문제목 있을 경우만
        try :
            title = i.find('div','cont').find('p','title').get_text()
        except :
            continue
        else :
            print('1.번호:',no)
            no2.append(no)

            print('2.논문제목:',title)
            title2.append(title)
            
            # 저자 정보
            all_data = i.find('p','etc').find_all('span')
            try :
                author=all_data[0].get_text().strip()
            except :
                author = '저자 정보가 없습니다'
                print("3.저자정보 : %s" %author)
                author2.append(author)
            else :
                author2.append(author)
                print("3.저자정보 : %s" %author)

            # 소속기관
            try :
                company=all_data[1].get_text().strip()
            except :
                company='소속 기관이 없습니다'
                company2.append(company)
                print("4.소속기관 : %s" %company)
            else :
                company2.append(company)
                print("4.소속기관 : %s" %company)
                
            # 발표년도
            try :
                date_2 = all_data[2].get_text().strip()
            except :
                date_2='발표년도가 없습니다'
                date2.append(date_2)
                print("5.발표년도 : %s" %date_2)
            else :
                date2.append(date_2)
                print("5.발표년도 : %s" %date_2)
                
            # 학위정보
            try :
                hak = all_data[3].get_text().strip()
            except :
                hak='학위정보가 없습니다'
                hak2.append(hak)
                print("6.학위정보 : %s" %hak)
            else :
                hak2.append(hak)
                print("6.학위정보 : %s" %hak)
            
            no += 1
            print("\n")
            
            if no > collect_cnt :
                break

            time.sleep(1)        # 페이지 변경 전 1초 대기 

    a += 1 
    b = str(a)

    try :
        driver.find_element(By.LINK_TEXT ,'%s' %b).click() 
    except :
        driver.find_element(By.LINK_TEXT, ('다음 페이지로')).click()
        
print("요청하신 작업이 모두 완료되었습니다")


검색하신 키워드 전염병 (으)로 총 1,603 건의 학위논문이 검색되었습니다
